In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [7]:
data = pd.read_csv('processed_data.csv')
data.sample(n=5)

,price,bedrooms,bathrooms,living_square,total_square,floors,yr_built,yr_renovated
4806,51937600.0,4,3.50,35053.763441,1.189022e+06,2.0,17,17
18008,43947200.0,5,3.00,39462.365591,1.032258e+05,1.0,60,60
2451,31416800.0,3,2.00,18494.623656,8.758065e+04,2.0,66,66
8210,39879360.0,3,2.50,24193.548387,9.930108e+04,2.0,29,29
2258,38499200.0,4,2.75,26344.086022,1.613118e+05,1.0,40,40


In [8]:
y = data['price']
features = [
            'bedrooms', 
            'bathrooms', 
            'living_square', 
            'total_square', 
            'floors',
            'yr_built',
            'yr_renovated',
           ]
categorical_features = []
X = data[features]
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)

train_pool = Pool(train_X, 
                  train_y, 
                  cat_features=categorical_features)
test_pool = Pool(test_X, 
                 cat_features=categorical_features)

In [9]:
model = CatBoostRegressor(
                              depth=10, 
                              iterations=5000, 
                              learning_rate=0.03, 
                              loss_function='MAE'
                         )
model.fit(train_pool, verbose=False)

In [10]:
preds = model.predict(test_pool)
print('Mean: ', (abs(preds - test_y) / test_y * 100).mean())
print('Median: ', (abs(preds - test_y) / test_y * 100).median())
print('Max: ', (abs(preds - test_y) / test_y * 100).max())
print('Min: ', (abs(preds - test_y) / test_y * 100).min())

Mean:  27.22301142576024
Median:  20.356276002946032
Max:  301.6790063072405
Min:  5.816060096627053e-05


In [11]:
import pickle
with open('Houses.pkl', 'wb') as model_file:
  pickle.dump(model, model_file)